In [75]:
import json
from web3 import Web3
import pandas as pd
%precision 30

'%.30f'

In [13]:
class web3kit:
    def connect(self, node='infura', network='mainnet', connection='HTTP'):
        """
        Node: geth, infura
        Network: mainnet, ropsten, rinkeby...
        Connection: IPC, Websocket, HTTP
        """
        
        if connection == 'IPC':
            if node == 'infura':
                    raise ValueError('Infure not available with connection type IPC')
            else:
                if network == 'mainnet':
                        self.web3 = Web3(Web3.IPCProvider('~/.ethereum/geth.ipc'))
                else:
                    self.web3 = Web3(Web3.IPCProvider('~/.ethereum/geth' + network + '/geth.ipc'))

        elif connection == 'Websocket':
            if node == 'infura':
                with open('/mnt/c/blockchain/api_keys/infura.json') as f:
                    self.infdata = json.load(f)
                self.infurl = "wss://" + network + ".infura.io/ws/v3/" + self.infdata['web3_eth1']['project_id']
                self.web3 = Web3(Web3.WebsocketProvider(self.infurl))
            else:
                raise ValueError('Geth websocket not yet implemented')
        
        elif connection == 'HTTP':
            if node == 'infura':
                with open('/mnt/c/blockchain/api_keys/infura.json') as f:
                    self.infdata = json.load(f)
                self.infurl = "https://" + network + ".infura.io/v3/" + self.infdata['web3_eth1']['project_id']
                self.web3 = Web3(Web3.HTTPProvider(self.infurl))
            else:
                raise ValueError('Geth HTTP not yet implemented')
        else:
            raise ValueError('Invalid connection type')
            
        return self.web3


    def get_balance(self, token, public_address, contract_address_json, ABI, currency_quote):
        """
        Get the balance of a particular token
        """
        if token == 'ether' or 'ETH':
            self.wei = web3.eth.getBalance(public_address)

        else:
            self.cpubkey = Web3.toChecksumAddress(public_address)
            
            try:
                self.contract_address_json = contract_address_json[token]
            except Exception:
                raise ValueError('Coin ABI not in ABI json')
            self.ccontractaddy = Web3.toChecksumAddress(self.contract_address_json)
            self.contract = web3.eth.contract(self.ccontractaddy, abi=ABI)
            self.wei = self.contract.functions.balanceOf(self.cpubkey).call()
        self.balance = float(web3.fromWei(self.wei, currency_quote))
        return self.balance


In [14]:
node = 'infura'  # geth, infura
network = 'ropsten' # mainnet, ropsten, rinkeby, polygon-mainnet
connection = 'HTTP' # HTTP, IPC, Websocket
token = 'ether' # ETH, USDT, USDC, etc...
currency_quote = 'ether'

web3 = web3kit().connect(node, network, connection)
print(web3.isConnected())


True


In [59]:
with open('/mnt/c/blockchain/api_keys/metamask.json') as f:
        maskdata = json.load(f)
pubkey = maskdata['account1']['public']
privkey = maskdata['account1']['private']

pubkey2 = '0x83b8E4C91Cd770F2ef10105A5c00BC7b94279087'

with open("./contract_address.json") as f:
    contractaddy = json.load(f)

with open("./ABI.json") as f:
    ABI = json.load(f)

balance = web3kit().get_balance(token, pubkey, contractaddy, ABI, currency_quote)

In [31]:
balance

5.299633868997071

In [32]:
nonce = web3.eth.getTransactionCount(pubkey)
tx = {
    'nonce': nonce,
    'to': pubkey2,
    'value': web3.toWei(.1, 'ether'),
    'gas': 5000000,
    'gasPrice': web3.toWei('50', 'gwei') # web3.eth.getGasPrice()  find alternative to js function
}

signed_tx = web3.eth.account.signTransaction(tx, privkey)
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
transaction_data = web3.eth.get_transaction(tx_hash)
receipt = web3.eth.get_transaction_receipt(tx_hash)

In [91]:
final_fee = web3.fromWei(receipt['gasUsed'], 'ether')

value = web3.fromWei(web3.eth.get_transaction(tx_hash)['value'], 'ether') # can also just get transaction value from tx

actual_cost = value + final_fee

Decimal('0.100000000000021')

In [63]:
# Use for gas estimates
# gasprice_wei = web3.fromWei(tx['gasPrice'], 'ether')
# fee =  #gas limit# * gasprice_wei   # in ether
# final_value = float(web3.fromWei(tx['value'], 'ether')) - fee

# actual_fee = (balance1 - balance2 - web3.from_wei(tx['value'], 'ether'))
# actual_gas = actual_fee / web3.fromWei(tx['gasPrice'])

AttributeDict({'blockHash': HexBytes('0xffc9e0ac1f5d9de9f7212aeddcc5237f50154aca51f57bebff115911d2f1ca00'),
 'blockNumber': 11276696,
 'from': '0xDB957d801E7b1a53D8E932f6D302012A25D8388B',
 'gas': 5000000,
 'gasPrice': 50000000000,
 'hash': HexBytes('0x1ef16afd8fcd154cd990ab87d8cd1c288ed5bf89b32707af6277d2847dbfd636'),
 'input': '0x',
 'nonce': 1,
 'r': HexBytes('0x90d44403cc945e9c087dcd80406e4045d246feae0738b3a99979af2b6b7c2961'),
 's': HexBytes('0x4e7f286bb174872806ca8235ed6f91cb533551870f0b3323ce0b0ad9519b5d72'),
 'to': '0x83b8E4C91Cd770F2ef10105A5c00BC7b94279087',
 'transactionIndex': 0,
 'type': '0x0',
 'v': 27,
 'value': 100000000000000000})

AttributeDict({'blockHash': HexBytes('0xffc9e0ac1f5d9de9f7212aeddcc5237f50154aca51f57bebff115911d2f1ca00'),
 'blockNumber': 11276696,
 'contractAddress': None,
 'cumulativeGasUsed': 21000,
 'effectiveGasPrice': 50000000000,
 'from': '0xDB957d801E7b1a53D8E932f6D302012A25D8388B',
 'gasUsed': 21000,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0x83b8E4C91Cd770F2ef10105A5c00BC7b94279087',
 'transactionHash': HexBytes('0x1ef16afd8fcd154cd990ab87d8cd1c288ed5bf89b32707af